In [14]:
import subprocess
import numpy as np
import re

J'ai seuelement effectué pour l'abre de décision, régression linéaire et random forest

In [15]:
def get_output(r_script = "reg.R"):
    process = subprocess.Popen(["Rscript", r_script], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    stdout, stderr = process.communicate()
    output = stdout.strip().split("\n")
    #print(stderr)
    return output

In [16]:
percentage_times_attribute_appears = 0.5

In [17]:
def random_forest(times = 1):
    attributes_boot = []
    error_rate_boot = []

    for b in range(times):
        output = get_output(r_script = "random_2.R")
        error_rate_boot.append(float(output[-1][4:]))
        
        top_fetaures_to_keep = 5
        for i in range(0, top_fetaures_to_keep*2, 2):
            attribute = output[i].strip()
            #print(attribute)
            attributes_boot.append(attribute)

    attributes_keep = []
    for attribute in set(attributes_boot):
        if(attributes_boot.count(attribute)/times>percentage_times_attribute_appears):
            attributes_keep.append(attribute)

    attributes_keep.sort()
    return np.array(error_rate_boot).mean(), attributes_keep

#random_forest(0)

In [18]:
def cart(times = 1):
    attributes_boot = []
    error_rate_boot = []

    for b in range(times):
        output = get_output(r_script = "cart_2.R")
        error_rate_boot.append(float(output[-1][4:]))
        temp = []
        for match in re.finditer(r'\d+\) (\w+)(<|>)', '\n'.join(output)):
            temp.append(match.group(1))
        temp = list(set(temp))
        attributes_boot+=temp

        
    attributes_keep = []
    for attribute in set(attributes_boot):
        if(attributes_boot.count(attribute)/times>percentage_times_attribute_appears):
            attributes_keep.append(attribute)

    attributes_keep.sort()
    return np.array(error_rate_boot).mean(), attributes_keep

#error_rate, attributes = cart(0)

In [19]:
def regression(times = 0):
    attributes_boot = []
    error_rate_boot = []

    for b in range(times):
        output = get_output(r_script = "reg_2.R")
        for i in range(0, len(output), 2):
            try:
                attribute = output[i].strip()
                pval = float(output[i+1].strip())
                if(pval<0.05 and attribute!='(Intercept)'):
                    attributes_boot.append(attribute)
                    #print(attribute, pval)
            except:
                error_rate = float(output[i].strip())
                error_rate_boot.append(error_rate)

    attributes_keep = []
    for attribute in set(attributes_boot):
        if(attributes_boot.count(attribute)/times>percentage_times_attribute_appears):
            attributes_keep.append(attribute)

    attributes_keep.sort()
    return np.array(error_rate_boot).mean(), attributes_keep

In [20]:
times = 200
_, regression_attributes = regression(times)
_, cart_attributes = cart(times)
_, random_forest_attributes = random_forest(times)


## Results of best attributes

In [21]:
print("regression_attributes", regression_attributes)
print("cart_attributes", cart_attributes)
print("random_forest_attributes", random_forest_attributes)


regression_attributes ['Input_number_similarity', 'Output_number_similarity', 'Output_type_jaccard', 'Rule_name_levenshtein', 'Shell_tools_categories_bioweb_jaccard', 'Shell_tools_description_biotools_levenshtein', 'Shell_tools_description_biotools_no_white_space_levenshtein', 'Shell_tools_function_biotools_jaccard', 'Shell_tools_operations_bioweb_jaccard', 'Shell_tools_topics_biotools_jaccard']
cart_attributes ['Conda_name_before_extension_levenshtein', 'Conda_no_white_spaces_cosine', 'Extracted_raw_rules_levenshtein', 'Input_number_similarity', 'Input_type_jaccard', 'Output_type_jaccard', 'Raw_rules_no_white_spaces_cosine', 'Raw_rules_no_white_spaces_levenshtein', 'Rule_name_levenshtein', 'Shell_tools_categories_bioweb_jaccard', 'Shell_tools_description_biotools_cosine', 'Shell_tools_description_biotools_levenshtein', 'Shell_tools_description_biotools_no_white_space_cosine', 'Shell_tools_description_bioweb_cosine', 'Shell_tools_description_bioweb_levenshtein', 'Shell_tools_topics_bio

In [22]:
def find_common_elements_at_least_3(list1, list2, list3):
    # Use set intersection to find common elements
    common_elements = set(list1) & set(list2) & set(list3)
    
    # Convert the set back to a list
    common_elements_list = list(common_elements)
    
    return common_elements_list


def find_common_elements_at_least_2(list1, list2, list3):
    # Use set intersection to find common elements
    common_elements = list(set(list1) & set(list2))
    common_elements += list(set(list1) & set(list3))
    common_elements += list(set(list2) & set(list3))    
    # Convert the set back to a list
    common_elements_list = list(set(common_elements))
    
    return common_elements_list

find_common_elements_at_least_3(regression_attributes, cart_attributes, random_forest_attributes)

['Output_type_jaccard', 'Rule_name_levenshtein']

In [23]:
results = {}
times = 200

results["reg"] = {}
results["cart"] = {}
results["random"] = {}

results["reg"]['ER'] = []
results["reg"]['sensitivity_binary'] = []

results["cart"]['ER'] = []
results["cart"]['sensitivity_D'] = []
results["cart"]['sensitivity_SS'] = []
results["cart"]['sensitivity_S'] = []
results["cart"]['sensitivity_VS'] = []

results["random"]['ER'] = []
results["random"]['sensitivity_D'] = []
results["random"]['sensitivity_SS'] = []
results["random"]['sensitivity_S'] = []
results["random"]['sensitivity_VS'] = []

for i in range(times):
    process = subprocess.Popen(["Rscript", "test_2.R"] + find_common_elements_at_least_3(regression_attributes, cart_attributes, random_forest_attributes), stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    stdout, stderr = process.communicate()
    for match in re.finditer(r"(\w+) +(\w+) +(\d(\.\d+)?)", stdout):
        model = match.group(1)
        stat = match.group(2)
        val = float(match.group(3))
        results[model][stat].append(val)
      

## Error rate and sensitivity

In [24]:
for model in results:
    print(model)
    for stat in results[model]:
        print(f"- {stat} {np.array(results[model][stat]).mean()*100:.2f}%")
    print()

reg
- ER 40.61%
- sensitivity_binary 77.55%

cart
- ER 26.32%
- sensitivity_D 82.23%
- sensitivity_SS 46.73%
- sensitivity_S 68.89%
- sensitivity_VS 97.03%

random
- ER 31.33%
- sensitivity_D 93.43%
- sensitivity_SS 28.93%
- sensitivity_S 54.78%
- sensitivity_VS 97.79%

